# Clustering


In [1]:
#import libraries

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS

In [2]:
import os
emails = pd.read_csv('enron_emailsdf.csv')
print(emails.shape)
emails.head()

(224128, 12)


,Unnamed: 0,Message-ID,Date,From,To,Subject,Path,Origin,FileName,Content,Content Length,Reply
0,0,<29790972.1075855665306.JavaMail.evans@thyme>,2000-12-13 18:41:00-08:00,1.11913372.-2@multexinvestornetwork.com,pallen@enron.com,"December 14, 2000 - Bear Stearns' predictions ...",\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,In today's Daily Update you'll find free repor...,8486,0
1,1,<21975671.1075855665520.JavaMail.evans@thyme>,2000-12-13 08:35:00-08:00,messenger@ecm.bloomberg.com,NaN,Bloomberg Power Lines Report,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,Here is today's copy of Bloomberg Power Lines....,16252,0
2,2,<7452188.1075855667684.JavaMail.evans@thyme>,2000-10-09 07:16:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0
3,3,<23790115.1075855667708.JavaMail.evans@thyme>,2000-10-09 07:00:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0
4,4,<5860470.1075855667730.JavaMail.evans@thyme>,2000-10-05 06:26:00-07:00,phillip.allen@enron.com,david.delainey@enron.com,NaN,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,"Dave, \n\n Here are the names of the west desk...",120,0


In [3]:
#make smaller chunks of emails
out = None
for emails in pd.read_csv('enron_emailsdf.csv', chunksize = 10000):
    if out is None:
        out = emails.copy()
    else:
        out.append(emails)
    out.head()

/var/folders/tv/54gx2c8n5xlc8cb4nmj9lywh0000gn/T/ipykernel_57425/3061489786.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out.append(emails)
/var/folders/tv/54gx2c8n5xlc8cb4nmj9lywh0000gn/T/ipykernel_57425/3061489786.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out.append(emails)
/var/folders/tv/54gx2c8n5xlc8cb4nmj9lywh0000gn/T/ipykernel_57425/3061489786.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out.append(emails)
/var/folders/tv/54gx2c8n5xlc8cb4nmj9lywh0000gn/T/ipykernel_57425/3061489786.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out.append(emails)
/var/folders/tv/54gx2c8n5xlc8cb4nmj9lywh0000gn/T/ipykern

In [4]:
print(out.shape)

(10000, 12)


In [5]:
print (out.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      10000 non-null  int64 
 1   Message-ID      10000 non-null  object
 2   Date            10000 non-null  object
 3   From            10000 non-null  object
 4   To              9643 non-null   object
 5   Subject         9061 non-null   object
 6   Path            9997 non-null   object
 7   Origin          9997 non-null   object
 8   FileName        9997 non-null   object
 9   Content         10000 non-null  object
 10  Content Length  10000 non-null  int64 
 11  Reply           10000 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 937.6+ KB
None


In [ ]:
# extract to, from and body from each email

def email_parsing(raw_message):
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

In [ ]:
# compile all extracted data into a single dictionary

def emails_parsing(messages):
    emails = [email_parsing(message) for message in messages]
    return {
        'body': result_append(emails, 'body'), 
        'to': result_append(emails, 'to'), 
        'from_': result_append(emails, 'from')
    }

In [ ]:
# append all key results into single list
def result_append(emails, key):
    results = []
    for email in emails:
        if key not in email:
            results.append('')
        else:
            results.append(email[key])
    return results

In [ ]:
# create a new dataframe for extracted data
extracted_data = pd.DataFrame(emails_parsing(out['Content']))

In [ ]:
# check any NaN values
extracted_data.isnull().any()

In [ ]:
# drop empty values
# extracted_data.drop(extracted_data.query("body == '' | to == '' | from_ == ''").index, inplace = True)

stopwords = ENGLISH_STOP_WORDS.union(['ect', 'hou', 'com', 'recipient'])
vect = TfidfVectorizer(analyzer = 'word', stop_words = stopwords, max_df = 0.5, min_df = 2)

In [ ]:
X = vect.fit_transform(extracted_data.body)

In [ ]:
X_dense = X.todense()
coords = PCA(n_components=2).fit_transform(X_dense)
plt.scatter(coords[:, 0], coords[:, 1], c = 'm')

In [ ]:
features = vect.get_feature_names()

In [ ]:
def top_msg_features(X, features, row_id, top_words = 25):
    row = np.squeeze(X[row_id].toarray())
    return top_tfidf_features(row, features, top_words)

In [ ]:
def top_tfidf_features(row, features, top_words = 20):
	# argsort produces the indices that orders the row by tf-idf value
	# reverse them (into descending order) and select the top words
    top_ids = np.argsort(row)[::-1][:top_words]
    top_features = [(features[i], row[i]) for i in top_ids]
    df = pd.DataFrame(top_features, columns=['features', 'score'])
    return df

In [ ]:
def top_msg_features(X, features, row_id, top_words = 25):
    row = np.squeeze(X[row_id].toarray())
    return top_tfidf_features(row, features, top_words)

def top_terms(X, features, grp_ids = None, min_tfidf = 0.1, top_words = 25):
    if grp_ids:
        msgs = X[grp_ids].toarray()
    else:
        msgs = X.toarray()
    msgs[msgs < min_tfidf] = 0
    
    # calculate the mean of each column across the selected rows (which results in a single row of tf-idf values)
    tfidf_means = np.mean(msgs, axis = 0)
    
    # then pass on to the previous func for picking out the top n words
    return top_tfidf_features(tfidf_means, features, top_words)

## K-means

In [ ]:
# create classifier with 4 clusters and 100 iterations
clf = KMeans(n_clusters = 4, max_iter = 100, init = 'k-means++', n_init = 1)
labels = clf.fit_predict(X)

In [ ]:
X_dense = X.todense()
pca = PCA(n_components = 2).fit(X_dense)
coords = pca.transform(X_dense)

In [ ]:
label_colours = ['b', 'm', 'r']
colours = [label_colours[i] for i in labels]
plt.scatter(coords[:, 0], coords[:, 1], c = colours)

In [ ]:
def top_words_per_cluster(X, y, features, min_tfidf = 0.1, top_words = 25):
    dfs = []

    labels = np.unique(y)
    for label in labels:
        ids = np.where(y == label) 
        features_df = top_terms(X, features, ids, min_tfidf = min_tfidf, top_words = top_words)
        features_df.label = label
        dfs.append(features_df)
    return dfs

In [ ]:
def plot_top_words(dfs):
    fig = plt.figure(figsize = (11, 8), facecolor = "w")
    x = np.arange(len(dfs[0]))
    for i, df in enumerate(dfs):
        ax = fig.add_subplot(1, len(dfs), i+1)
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.set_frame_on(False)
        ax.get_xaxis().tick_bottom()
        ax.get_yaxis().tick_left()
        ax.set_xlabel("Tf-Idf Score", labelpad = 16, fontsize = 13)
        ax.set_title("cluster = " + str(df.label), fontsize = 15)
        ax.ticklabel_format(axis = 'x', style = 'sci', scilimits = (-2, 2))
        ax.barh(x, df.score, align = 'center', color = '#7530FF')
        ax.set_yticks(x)
        ax.set_ylim([-1, x[-1] + 1])
        yticks = ax.set_yticklabels(df.features)
        plt.subplots_adjust(bottom = 0.09, right = 0.97, left = 0.15, top = 0.95, wspace = 0.52)
    plt.show()

In [ ]:
tops = top_words_per_cluster(X, labels, features, 0.1, 25)

In [ ]:
plot_top_words(tops)